# Overview
This is setup Jupyter notebook to generate initial testing transactions.

Note to be able to generate all transactions hashes you need the following:
* run localnet via `make clean debug-multi-bls`
* wait for on the epoch 2+, scripts below have a check for this
* run steps below one by one subsequently, because of increasing nonce

Transactions, for the signed transition and transition's hash go to sections output below:

| # | Action                  | From / To                                                                 |
|---|-------------------------|---------------------------------------------------------------------------|
| 1 | Simple Transfer         | `one155jp2y76nazx8uw5sa94fr0m4s5aj8e5xm6fu3` to `one1ru3p8ff0wsyl7ncsx3vwd5szuze64qz60upg37` |
| 2 | Contract Creation       | `one1ru3p8ff0wsyl7ncsx3vwd5szuze64qz60upg37`                              |
| 3 | Cross-Shard Transfer    | `one1ru3p8ff0wsyl7ncsx3vwd5szuze64qz60upg37` to `one1e8rdglh97t37prtnv7k35ymnh2wazujpzsmzes` |
| 4 | Validator Creation      | By `one155jp2y76nazx8uw5sa94fr0m4s5aj8e5xm6fu3`                            |
| 5 | Delegation              | From `one1ru3p8ff0wsyl7ncsx3vwd5szuze64qz60upg37` to `one155jp2y76nazx8uw5sa94fr0m4s5aj8e5xm6fu3` |
| 6 | EditValidator out of scope | By `one155jp2y76nazx8uw5sa94fr0m4s5aj8e5xm6fu3` |
| 7 | Collect rewards by      | From `one155jp2y76nazx8uw5sa94fr0m4s5aj8e5xm6fu3` to `one1ru3p8ff0wsyl7ncsx3vwd5szuze64qz60upg37` |
| 8 | Undelegation            | From `one1ru3p8ff0wsyl7ncsx3vwd5szuze64qz60upg37` to `one155jp2y76nazx8uw5sa94fr0m4s5aj8e5xm6fu3` |

## 1. Simple Transfer

In [1]:
from pyhmy import signing, numbers, transaction, blockchain
from pyhmy.rpc.request import RPCError
from web3 import Web3

rpc_endpoint = "http://localhost:9620"
pk = "1f84c95ac16e6a50f08d44c7bde7aff8742212fda6e4321fde48bf83bef266dc"

def is_epoch_at_least(endpoint, min_epoch):
    try:
        header = blockchain.get_latest_header(endpoint=endpoint)
        current_epoch = int(header['epoch'])
        return current_epoch >= min_epoch
    except Exception as e:
        print(f"Error checking epoch: {e}")
        return False

tx = {
    'from': 'one155jp2y76nazx8uw5sa94fr0m4s5aj8e5xm6fu3',
    'to': 'one1ru3p8ff0wsyl7ncsx3vwd5szuze64qz60upg37',
    'gasPrice': Web3.to_wei(100, 'gwei'),
    'gas': 21000,
    'chainId': 2,  # localnet
    'value': int(numbers.convert_one_to_atto(503)),
    'nonce': 0,
    'shardID': 0,
    'toShardID': 0,
}

raw_tx = signing.sign_transaction(tx, pk).raw_transaction.to_0x_hex()
print(f"Raw TX: {raw_tx}")

if not is_epoch_at_least(rpc_endpoint, 2):
    print("Localnet has not reached the required epoch (2). Transaction will NOT be submitted and printed out")
    exit(1)
else:
    try:
        tx_hash = transaction.send_raw_transaction(raw_tx, endpoint=rpc_endpoint)
        print(f"Transaction sent: {tx_hash}")
    except RPCError as e:
        if "transaction already finalized" in str(e):
            print("Transaction already finalized, skipping resend, please restart localnet to get it")
            tx_hash = None  # or handle appropriately
        else:
            raise


Raw TX: 0xf86f8085174876e8008252088080941f2213a52f7409ff4f103458e6d202e0b3aa805a891b4486fafde57c00008027a0d7c0b20207dcc9dde376822dc3f5625eac6f59a7526111695cdba3e29553ca17a05d4ca9a421ae16f89cbf6848186eaea7a800da732446dff9952e7c1e91d414e3
Transaction sent: 0xc26be5776aa57438bccf196671a2d34f3f22c9c983c0f844c62b2fb90403aa43


## 2. Contract Creation

In [3]:
from pyhmy import signing, transaction, blockchain
from pyhmy.rpc.request import RPCError
from web3 import Web3

rpc_endpoint = "http://localhost:9620"
pk = "3c86ac59f6b038f584be1c08fced78d7c71bb55d5655f81714f3cddc82144c65"

def is_epoch_at_least(endpoint, min_epoch):
    try:
        header = blockchain.get_latest_header(endpoint=endpoint)
        current_epoch = int(header['epoch'])
        return current_epoch >= min_epoch
    except Exception as e:
        print(f"Error checking epoch: {e}")
        return False

data = "0x6080604052348015600f57600080fd5b50607780601d6000396000f3fe6080604052348015600f57600080fd5b506004361060285760003560e01c80634936cd3614602d575b600080fd5b604080516001815290519081900360200190f3fea2646970667358221220fa3fa0e8d0267831a59f4dd5edf39a513d07e98461cb06660ad28d4beda744cd64736f6c634300080f0033"

tx = {
    'from': 'one1ru3p8ff0wsyl7ncsx3vwd5szuze64qz60upg37',
    'gasPrice': Web3.to_wei(100, 'gwei'),
    'gas': 100000,
    'chainId': 2,
    'nonce': 0,
    'shardID': 0,
    'toShardID': 0,
    'data': data,
}

raw_tx = signing.sign_transaction(tx, pk).raw_transaction.to_0x_hex()
print(f"Raw TX: {raw_tx}")

if not is_epoch_at_least(rpc_endpoint, 2):
    print("Localnet has not reached the required epoch (2). Transaction will NOT be submitted and printed out")
    exit(1)
else:
    try:
        tx_hash = transaction.send_raw_transaction(raw_tx, endpoint=rpc_endpoint)
        print(f"Transaction sent: {tx_hash}")
    except RPCError as e:
        if "transaction already finalized" in str(e):
            print("Transaction already finalized, skipping resend. Restart localnet to retry cleanly")
        else:
            raise


Raw TX: 0xf8e88085174876e800830186a080808080b8946080604052348015600f57600080fd5b50607780601d6000396000f3fe6080604052348015600f57600080fd5b506004361060285760003560e01c80634936cd3614602d575b600080fd5b604080516001815290519081900360200190f3fea2646970667358221220fa3fa0e8d0267831a59f4dd5edf39a513d07e98461cb06660ad28d4beda744cd64736f6c634300080f003327a08bf26ee0120c296b17af507f62606abdb5c5f09a65642c3d30b349b8bfbb3d69a03ec7be51c615bcbf2f1d63f6eaa56cf8d7be81671717f90239619830a81ebc9f
Transaction sent: 0xa605852dd2fa39ed42e101c17aaca9d344d352ba9b24b14b9af94ec9cb58b31f


## 3.Cross Shard Transfer

In [4]:
from pyhmy import signing, numbers, transaction, blockchain
from pyhmy.rpc.request import RPCError
from web3 import Web3

rpc_endpoint = "http://localhost:9620"
pk = "3c86ac59f6b038f584be1c08fced78d7c71bb55d5655f81714f3cddc82144c65"

def is_epoch_at_least(endpoint, min_epoch):
    try:
        header = blockchain.get_latest_header(endpoint=endpoint)
        current_epoch = int(header['epoch'])
        return current_epoch >= min_epoch
    except Exception as e:
        print(f"Error checking epoch: {e}")
        return False

tx = {
    'from': "one1ru3p8ff0wsyl7ncsx3vwd5szuze64qz60upg37",
    'to': 'one1e8rdglh97t37prtnv7k35ymnh2wazujpzsmzes',
    'gasPrice': Web3.to_wei(100, 'gwei'),
    'gas': 21000,
    'chainId': 2,
    'nonce': 1,
    'shardID': 0,
    'toShardID': 1,
    'value': Web3.to_wei(100, 'gwei'),
}

raw_tx = signing.sign_transaction(tx, pk).raw_transaction.to_0x_hex()
print(f"Raw TX: {raw_tx}")

if not is_epoch_at_least(rpc_endpoint, 2):
    print("Localnet has not reached the required epoch (2). Transaction will NOT be submitted and printed out")
    exit(1)
else:
    try:
        tx_hash = transaction.send_raw_transaction(raw_tx, endpoint=rpc_endpoint)
        print(f"Transaction sent: {tx_hash}")
    except RPCError as e:
        if "transaction already finalized" in str(e):
            print("Transaction already finalized, skipping resend. Restart localnet to retry cleanly")
        else:
            raise



Raw TX: 0xf86b0185174876e800825208800194c9c6d47ee5f2e3e08d7367ad1a1373ba9dd1724185174876e8008027a02501c517220e9499f14e97c20b0a88cd3b7ba80637bba43ed295422e69a3f300a079b8e1213c9506184aed6ac2eb0b2cb00594c3f9fcdd6c088937ce17fe47107c
Transaction sent: 0xf73ba634cb96fc0e3e2c9d3b4c91379e223741be4a5aa56e6d6caf49c1ae75cf


## 4.Validator Creation
## 5. Delegation
Should be done together in the neighbor blocks

In [5]:
from pyhmy import signing, numbers, transaction, blockchain, account, validator as validator_module, staking_signing, staking_structures
from pyhmy.rpc.request import RPCError
from web3 import Web3
import time
import random
import requests
import json

rpc_endpoint = "http://localhost:9620"
pk_validator = "1f84c95ac16e6a50f08d44c7bde7aff8742212fda6e4321fde48bf83bef266dc"
pk_delegator = "3c86ac59f6b038f584be1c08fced78d7c71bb55d5655f81714f3cddc82144c65"

validator_address = 'one155jp2y76nazx8uw5sa94fr0m4s5aj8e5xm6fu3'
delegator_address = 'one1ru3p8ff0wsyl7ncsx3vwd5szuze64qz60upg37'
validator_nonce = 1
delegate_nonce = 2

info = {
    "name": "Alice",
    "identity": "alice",
    "website": "alice.harmony.one",
    "security-contact": "Bob",
    "details": "Are you even reading this?",
    "min-self-delegation": int(numbers.convert_one_to_atto(10000)),
    "max-total-delegation": int(numbers.convert_one_to_atto(100000)),
    "rate": "0.1",
    "max-rate": "0.9",
    "max-change-rate": "0.05",
    "bls-public-keys": [
        "0xa20e70089664a874b00251c5e85d35a73871531306f3af43e02138339d294e6bb9c4eb82162199c6a852afeaa8d68712",
    ],
    "amount": int(numbers.convert_one_to_atto(10000)),
    "bls-key-sigs": [
        "0xef2c49a2f31fbbd23c21bc176eaf05cd0bebe6832033075d81fea7cff6f9bc1ab42f3b6895c5493fe645d8379d2eaa1413de55a9d3ce412a4f747cb57d52cc4da4754bfb2583ec9a41fe5dd48287f964f276336699959a5fcef3391dc24df00d",
    ]
}

def is_epoch_at_least(endpoint, min_epoch):
    try:
        header = blockchain.get_latest_header(endpoint=endpoint)
        current_epoch = int(header['epoch'])
        return current_epoch >= min_epoch
    except Exception as e:
        print(f"Error checking epoch: {e}")
        return False

def _check_staking_transaction(stx_hash, endpoint=rpc_endpoint, timeout=30):
    payload = {
        "id": "1",
        "jsonrpc": "2.0",
        "method": "hmyv2_getStakingTransactionByHash",
        "params": [stx_hash],
    }
    headers = {"Content-Type": "application/json"}
    response = requests.post(endpoint, headers=headers, data=json.dumps(payload), timeout=timeout)
    return response.json()

def wait_for_staking_transaction_confirmed(tx_hash, endpoint, timeout=30):
    start_time = time.time()
    while time.time() - start_time <= timeout:
        tx_data = _check_staking_transaction(tx_hash, endpoint)
        if tx_data and "result" in tx_data:
            block_hash = tx_data["result"].get("blockHash", "0x00")
            if set(block_hash[2:]) != {"0"}:
                return True
        time.sleep(random.uniform(0.3, 0.6))
    return False

def wait_for_validator_on_chain(validator_addr, endpoint, timeout=30):
    start = time.time()
    while time.time() - start < timeout:
        try:
            info = validator_module.get_validator_information(validator_addr, endpoint=endpoint)
            if info and "validator" in info and info["validator"]:
                return True
        except RPCError as e:
            if "not found address" not in str(e):
                raise
        time.sleep(random.uniform(0.5, 1.0))
    return False

# Step 1: Create Validator
validator = validator_module.Validator(validator_address)
validator.load(info)

raw_create_tx = validator.sign_create_validator_transaction(
    validator_nonce,
    Web3.to_wei(100, 'gwei'),
    55000000,
    pk_validator,
    2
).raw_transaction.to_0x_hex()

print(f"Raw CreateValidator TX: {raw_create_tx}")

if not is_epoch_at_least(rpc_endpoint, 2):
    print("Localnet has not reached the required epoch (2), transaction will NOT be submitted")
    exit(1)
else:
    try:
        tx_hash = transaction.send_raw_staking_transaction(raw_create_tx, endpoint=rpc_endpoint)
        print(f"Validator creation transaction sent: {tx_hash}")
        time.sleep(4)
        if not wait_for_staking_transaction_confirmed(tx_hash, rpc_endpoint, timeout=90):
            print("Validator transaction not confirmed in time — aborting")
            exit(1)
    except RPCError as e:
        error_msg = str(e)
        if "transaction already finalized" in error_msg:
            print("Transaction already finalized, skipping resend — restart localnet to retry cleanly")
        else:
            raise

# Step 2: Wait for validator to appear on chain
if not wait_for_validator_on_chain(validator_address, rpc_endpoint):
    print("Validator creation not finalized — cannot proceed with delegation")
    exit(1)

# Step 3: Delegate


delegation_tx = {
    'directive': staking_structures.Directive.Delegate,
    'delegatorAddress': delegator_address,
    'validatorAddress': validator_address,
    'amount': Web3.to_wei(100, 'ether'),
    'nonce': delegate_nonce,
    'gasPrice': Web3.to_wei(100, 'gwei'),
    'gasLimit': 50000,
    'chainId': 2,
}

raw_delegate_tx = staking_signing.sign_staking_transaction(delegation_tx, pk_delegator).raw_transaction.to_0x_hex()
print(f"Raw Delegate TX: {raw_delegate_tx}")

if not is_epoch_at_least(rpc_endpoint, 2):
    print("Localnet has not reached the required epoch (2), transaction will NOT be submitted")
    exit(1)
else:
    try:
        current_epoch = is_epoch_at_least(rpc_endpoint, 2)
        tx_hash = transaction.send_raw_staking_transaction(raw_delegate_tx, endpoint=rpc_endpoint)
        print(f"Delegation transaction sent: {tx_hash}")
    except RPCError as e:
        error_msg = str(e)
        if "transaction already finalized" in error_msg:
            print("Transaction already finalized, skipping resend — restart localnet to retry cleanly")
        elif "staking validator does not exist" in error_msg:
            print("Staking failed: Validator does not exist — ensure the validator creation transaction succeeded and is finalized")
        else:
            raise

Raw CreateValidator TX: 0xf9017c80f9012994a5241513da9f4463f1d4874b548dfbac29d91f34f83d85416c69636585616c69636591616c6963652e6861726d6f6e792e6f6e6583426f629a41726520796f75206576656e2072656164696e6720746869733fddc988016345785d8a0000c9880c7d713b49da0000c887b1a2bc2ec500008a021e19e0c9bab24000008a152d02c7e14af6800000f1b0a20e70089664a874b00251c5e85d35a73871531306f3af43e02138339d294e6bb9c4eb82162199c6a852afeaa8d68712f862b860ef2c49a2f31fbbd23c21bc176eaf05cd0bebe6832033075d81fea7cff6f9bc1ab42f3b6895c5493fe645d8379d2eaa1413de55a9d3ce412a4f747cb57d52cc4da4754bfb2583ec9a41fe5dd48287f964f276336699959a5fcef3391dc24df00d8a021e19e0c9bab24000000185174876e8008403473bc028a08c1146305eaef981aa24c2f17c8519664d10c99ee42acedbc258749930d31a7ca031dadf114ee6ab9bd09933208094c65037b66c796bcfc57a70158106b37357b0
Validator creation transaction sent: 0x400e9831d358f5daccd153cad5bf53650a0d413bd8682ec0ffad55367d162968
Raw Delegate TX: 0xf88302f4941f2213a52f7409ff4f103458e6d202e0b3aa805a94a5241513da9f4463f1d4874b548dfbac

## 6. EditValidator - out of scope rn

Reason - absence of the `epos-eligibility-status` field in the edit validator operation, this is causing `{'code': -32000, 'message': 'rlp: too few elements for types.EditValidator'}` error

Harmony code:
https://github.com/harmony-one/harmony/blob/dc52025a8a1fb08539a9024f1c062859ca03a317/staking/types/messages.go#L103-L113

In [66]:
from pyhmy import signing, numbers, transaction, blockchain, account, validator as validator_module, staking_signing, staking_structures
from pyhmy.rpc.request import RPCError
from web3 import Web3
import time
import random
import requests
import json

rpc_endpoint = "http://localhost:9620"
pk_validator = "1f84c95ac16e6a50f08d44c7bde7aff8742212fda6e4321fde48bf83bef266dc"
pk_delegator = "3c86ac59f6b038f584be1c08fced78d7c71bb55d5655f81714f3cddc82144c65"

validator_address = 'one155jp2y76nazx8uw5sa94fr0m4s5aj8e5xm6fu3'
validator_nonce = 2

info = {
    "name": "AliceNEW",
    "identity": "alice",
    "website": "alice.harmony.one",
    "security-contact": "Bob",
    "details": "Are you even reading this?",
    "rate": "0.10",
    "max-rate": "0.9",
    "max-change-rate": "0.05",
    "security-contact": "Bob",
    "details": "Are you even reading this?",
    "min-self-delegation": int(numbers.convert_one_to_atto(10000)),
    "max-total-delegation": int(numbers.convert_one_to_atto(100000)),
    "bls-public-keys": [
        "0xa20e70089664a874b00251c5e85d35a73871531306f3af43e02138339d294e6bb9c4eb82162199c6a852afeaa8d68712",
    ],
    "amount": int(numbers.convert_one_to_atto(10000)),
    "bls-key-sigs": [
        "0xef2c49a2f31fbbd23c21bc176eaf05cd0bebe6832033075d81fea7cff6f9bc1ab42f3b6895c5493fe645d8379d2eaa1413de55a9d3ce412a4f747cb57d52cc4da4754bfb2583ec9a41fe5dd48287f964f276336699959a5fcef3391dc24df00d",
    ]
}

def is_epoch_at_least(endpoint, min_epoch):
    try:
        header = blockchain.get_latest_header(endpoint=endpoint)
        current_epoch = int(header['epoch'])
        return current_epoch >= min_epoch
    except Exception as e:
        print(f"Error checking epoch: {e}")
        return False


validator = validator_module.Validator(validator_address)
validator.load(info)

raw_create_tx = validator.sign_edit_validator_transaction(
    validator_nonce,
    Web3.to_wei(100, 'gwei'),
    55000000,
    "0.13",
    "0xa20e70089664a874b00251c5e85d35a73871531306f3af43e02138339d294e6bb9c4eb82162199c6a852afeaa8d68712",
    "0xb9486167ab9087ab818dc4ce026edb5bf216863364c32e42df2af03c5ced1ad181e7d12f0e6dd5307a73b62247608611",
    "0x68f800b6adf657b674903e04708060912b893b7c7b500788808247550ab3e186e56a44ebf3ca488f8ed1a42f6cef3a04bd5d2b2b7eb5a767848d3135b362e668ce6bba42c7b9d5666d8e3a83be707b5708e722c58939fe9b07c170f3b7062414",
    pk_validator,
    chain_id = 2,
).raw_transaction.to_0x_hex()

print(f"Raw EditValidator TX: {raw_create_tx}")

if not is_epoch_at_least(rpc_endpoint, 2):
    print("Localnet has not reached the required epoch (2), transaction will NOT be submitted")
    exit(1)
else:
    try:
        tx_hash = transaction.send_raw_staking_transaction(raw_create_tx, endpoint=rpc_endpoint)
        print(f"Validator edit transaction sent: {tx_hash}")
    except RPCError as e:
        error_msg = str(e)
        if "transaction already finalized" in error_msg:
            print("Transaction already finalized, skipping resend — restart localnet to retry cleanly")
        else:
            raise

Raw EditValidator TX: 0xf9018e01f9013b94a5241513da9f4463f1d4874b548dfbac29d91f34f84088416c6963654e455785616c69636591616c6963652e6861726d6f6e792e6f6e6583426f629a41726520796f75206576656e2072656164696e6720746869733fc98801cdda4faccd00008a021e19e0c9bab24000008a152d02c7e14af6800000b0a20e70089664a874b00251c5e85d35a73871531306f3af43e02138339d294e6bb9c4eb82162199c6a852afeaa8d68712b0b9486167ab9087ab818dc4ce026edb5bf216863364c32e42df2af03c5ced1ad181e7d12f0e6dd5307a73b62247608611b86068f800b6adf657b674903e04708060912b893b7c7b500788808247550ab3e186e56a44ebf3ca488f8ed1a42f6cef3a04bd5d2b2b7eb5a767848d3135b362e668ce6bba42c7b9d5666d8e3a83be707b5708e722c58939fe9b07c170f3b70624140285174876e8008403473bc027a0a0b9c9cb135216d4a12ae982d0157660507213bc5dfb47cec75bc978347d8f56a05d504c36a785edfe2432af1181e0907604ff9e406eba91720a95286652b68aba


RPCError: Error in reply from http://localhost:9620: hmyv2_sendRawStakingTransaction returned {'code': -32000, 'message': 'rlp: too few elements for types.EditValidator'}

## 7. Collect rewards

In [ ]:

from pyhmy import signing, numbers, transaction, blockchain, account, validator as validator_module, staking_signing, staking_structures
from pyhmy.rpc.request import RPCError
from web3 import Web3

rpc_endpoint = "http://localhost:9620"
pk_validator = "1f84c95ac16e6a50f08d44c7bde7aff8742212fda6e4321fde48bf83bef266dc"
pk_delegator = "3c86ac59f6b038f584be1c08fced78d7c71bb55d5655f81714f3cddc82144c65"

validator_address = 'one155jp2y76nazx8uw5sa94fr0m4s5aj8e5xm6fu3'
delegator_address = 'one1ru3p8ff0wsyl7ncsx3vwd5szuze64qz60upg37'
collect_reward_nonce = 3

undelegation_tx = {
    'directive': staking_structures.Directive.CollectRewards,
    'delegatorAddress': delegator_address,
    'amount': Web3.to_wei(100, 'ether'),
    'nonce': collect_reward_nonce,
    'gasPrice': Web3.to_wei(100, 'gwei'),
    'gasLimit': 50000,
    'chainId': 2,
}

raw_undelegate_tx = staking_signing.sign_staking_transaction(undelegation_tx, pk_delegator).raw_transaction.to_0x_hex()
print(f"Raw Сollect reward TX: {raw_undelegate_tx}")

if not is_epoch_at_least(rpc_endpoint, 2):
    print("Localnet has not reached the required epoch (2), transaction will NOT be submitted")
    exit(1)
else:
    try:
        tx_hash = transaction.send_raw_staking_transaction(raw_undelegate_tx, endpoint=rpc_endpoint)
        print(f"Collect reward transaction sent: {tx_hash}")
    except RPCError as e:
        error_msg = str(e)
        if "transaction already finalized" in error_msg:
            print("Transaction already finalized, skipping resend — restart localnet to retry cleanly")
        elif "no rewards to collect" in error_msg:
            print("no rewards to collect - normal behavior for now, should be okay in the the real test, for now as is")
        else:
            raise

Raw Сollect reward TX: 0xf86404d5941f2213a52f7409ff4f103458e6d202e0b3aa805a0385174876e80082c35028a07da4d36bf26f8d6b8895c5f1bbc0bf31e2e131f8903efe8f2ced9ca5fa20ef1fa01a9e6d671fa9a05c8ca20dc80a7f6ecce8e10fbe2beb2e3016eef5a37ba00020
no rewards to collect - normal behavior


## 8. Undelegate

In [11]:
from pyhmy import signing, numbers, transaction, blockchain, account, validator as validator_module, staking_signing, staking_structures
from pyhmy.rpc.request import RPCError
from web3 import Web3

rpc_endpoint = "http://localhost:9620"
pk_validator = "1f84c95ac16e6a50f08d44c7bde7aff8742212fda6e4321fde48bf83bef266dc"
pk_delegator = "3c86ac59f6b038f584be1c08fced78d7c71bb55d5655f81714f3cddc82144c65"

validator_address = 'one155jp2y76nazx8uw5sa94fr0m4s5aj8e5xm6fu3'
delegator_address = 'one1ru3p8ff0wsyl7ncsx3vwd5szuze64qz60upg37'
undelegate_nonce = 4

undelegation_tx = {
    'directive': staking_structures.Directive.Undelegate,
    'delegatorAddress': delegator_address,
    'validatorAddress': validator_address,
    'amount': Web3.to_wei(100, 'ether'),
    'nonce': undelegate_nonce,
    'gasPrice': Web3.to_wei(100, 'gwei'),
    'gasLimit': 50000,
    'chainId': 2,
}

raw_undelegate_tx = staking_signing.sign_staking_transaction(undelegation_tx, pk_delegator).raw_transaction.to_0x_hex()
print(f"Raw Undelegate TX: {raw_undelegate_tx}")

if not is_epoch_at_least(rpc_endpoint, 2):
    print("Localnet has not reached the required epoch (2), transaction will NOT be submitted")
    exit(1)
else:
    try:
        tx_hash = transaction.send_raw_staking_transaction(raw_undelegate_tx, endpoint=rpc_endpoint)
        print(f"Undelegation transaction sent: {tx_hash}")
    except RPCError as e:
        error_msg = str(e)
        if "transaction already finalized" in error_msg:
            print("Transaction already finalized, skipping resend — restart localnet to retry cleanly")
        elif "staking validator does not exist" in error_msg:
            print("Staking failed: Validator does not exist — ensure the validator creation transaction succeeded and is finalized")
        else:
            raise

Raw Undelegate TX: 0xf88303f4941f2213a52f7409ff4f103458e6d202e0b3aa805a94a5241513da9f4463f1d4874b548dfbac29d91f3489056bc75e2d631000000485174876e80082c35028a05e16bdc0938e57565be42955ed3fab407fceb7ce0f68716dcea4f7f7b26a7503a048fff237db207f890e7aa50de602e3eb525017884fb4f6e62da52165ac606e32
Undelegation transaction sent: 0xe4bc25428a463e88f49f138b5342864ea4e10edb0399022a61cc22debfba4f4f
